In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
import os
from os import listdir
from keras.models import Sequential

In [ ]:
#Data Augmentation using Python and Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        fill_mode="nearest")

In [2]:
img=load_img(r"C:\Users\ShibuKumar\Downloads\Dog\1.jpg")
x=img_to_array(img) #This is numpy array with shape (3,150,150)
x=x.reshape((1,) + x.shape) #This is a numpy array with shape (1,3, 150, 150);[150*150 pixel size, 3channel(RGB), 1 is array)

NameError: name 'load_img' is not defined

In [3]:
#.flow() command below generates batches of randomly transformed images. and saves the results to the preview/directory
i=0
for batch in datagen.flow(x,batch_size=1,save_to_dir=r"C:\Users\ShibuKumar\Downloads\Cat Preview",save_prefix="cat",save_format="jpeg"):
    i+=1
    if i>20:
        break #otherwise the generator would loop indefinitely

NameError: name 'datagen' is not defined

Create CNN Model and Optimize it using Keras Tuner

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [4]:
train_images[0].shape

(28, 28)

In [5]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [6]:
train_images[0].shape

(28, 28, 1)

In [7]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int("conv_1_Filter",min_value=32,max_value=128,step=16),
            kernel_size=hp.Choice("conv_1_kernel",values=[3,5]),
            activation="relu",
            input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int("conv_2_Filter",min_value=32,max_value=128,step=16),
            kernel_size=hp.Choice("con_2_kernel",values=[3,5]),
            activation="relu"
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int("dense_1_units",min_value=32,max_value=128,step=16),
            activation="relu"
        ),
        keras.layers.Dense(10,activation="softmax")
    ])

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate",values=[1e-2,1e-3])),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model    

In [1]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameter
tuner_search=RandomSearch(build_model,
                          objective="val_accuracy",
                          max_trials=5,
                          directory="output",
                          project_name="Mnist Fashion")

NameError: name 'build_model' is not defined

In [ ]:
tuner_search.search(train_images,train_labels,
                   epochs=3,validation_split=0.1)